In [1]:
import zipfile as zf
import requests
import xml.etree.ElementTree as ET
import pickle

In [2]:
# Guardar dicionarios com o pickle
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
print ('A fazer o download da base de dados do HMDB...')

u = requests.get("http://www.hmdb.ca/system/downloads/current/hmdb_metabolites.zip")
with open("hmdb_metabolites.zip", "wb") as f:
    f.write(u.content)

A fazer o download da base de dados do HMDB...


In [4]:
# Criar dicionário de conversão HMDB-KEGG
print ('A converter base de dados...')

conv={}
with zf.ZipFile('hmdb_metabolites.zip') as myzip:
    for x in myzip.namelist():
        if x != 'hmdb_metabolites.xml':
            with myzip.open(x) as myfile:
                a = ET.fromstring(myfile.read())[36].text
                if a != None and a.startswith('C'):
                    conv[x[:9]] = a[:6] # alguns códigos kegg aparecem com ' ' (espaço) no final, é necessário truncar
                else:
                    continue
print ('De '+str(len(zf.ZipFile('hmdb_metabolites.zip').namelist()))+' entradas, foram encontradas '+str(len(conv))+' equivalências (KEGG-HMDB)')

A converter base de dados...
De 41515 entradas, foram encontradas 7231 equivalências (KEGG-HMDB)


In [5]:
save_obj(conv, 'HMDB2KEGG')

In [6]:
# Encontrar o link do download
a = requests.get('http://lipidmaps.org/resources/downloads/index.html').text
b = a.splitlines()
for x in b:
    if '''.zip'>LMSDFDownload''' in x:
        r = x.split('''.zip'>''')[1].split('<')[0]

In [7]:
# Download da DB do LipipMaps
print ('')
print ('A fazer o download da base de dados do LipidMaps...')

u = requests.get("http://lipidmaps.org/resources/downloads/"+r)
with open(r, "wb") as f:
    f.write(u.content)

A fazer o download da base de dados do LipidMaps...


In [8]:
# Criar dicionário de conversão LipidMaps-KEGG
d={}
with zf.ZipFile(r) as myzip:
    for x in myzip.namelist():
        if x.endswith('All.sdf'):      
            with myzip.open(x) as myfile:
                a = bytes.decode(myfile.read())
                a = a.split('$$$$')
                for x in a:
                    x = x.split('\n\n')
                    for y in x:
                        if y.startswith('> <KEGG_ID>'):
                            if x[0].split('\n')[1].startswith('  LIPDMAPS'): # Este if garante que o 1º elemento também é adicionado corretamente
                                d[x[0].split('\n')[0]] = y[len('> <KEGG_ID>\n'):]
                            else:
                                d[x[0].split('\n')[1]] = y[len('> <KEGG_ID>\n'):]

In [9]:
# Criar dicionário de conversão KEGG-LipidMaps
d2={}
with zf.ZipFile(r) as myzip:
    for x in myzip.namelist():
        if x.endswith('All.sdf'):      
            with myzip.open(x) as myfile:
                a = bytes.decode(myfile.read())
                a = a.split('$$$$')
                for x in a:
                    x = x.split('\n\n')
                    for y in x:
                        if y.startswith('> <KEGG_ID>'):
                            if x[0].split('\n')[1].startswith('  LIPDMAPS'): # Este if garante que o 1º elemento também é adicionado corretamente
                                d2[y[len('> <KEGG_ID>\n'):]] = x[0].split('\n')[0]
                            else:
                                d2[y[len('> <KEGG_ID>\n'):]] = x[0].split('\n')[1]
print ('De '+str(len(a))+' entradas, foram encontradas '+str(len(d))+' equivalências (KEGG-LipidMaps)')

De 40720 entradas, foram encontradas 1489 equivalências (KEGG-LipidMaps)


In [10]:
save_obj(d, 'LM2KEGG')
save_obj(d2, 'KEGG2LM')